# Install the libraries 

In [11]:
%pip install PyPDF2
%pip install python-dotenv
%pip install -q -U google-generativeai



Note: you may need to restart the kernel to use updated packages.


# We import the necessary libraries

In [12]:
import os
import PyPDF2
from dotenv import load_dotenv 
import google.generativeai as genai

In [13]:
load_dotenv()

True

In [14]:
API_KEY_GOOGLE= os.getenv("API_KEY_GOOGLE_GEMINI")

# We read and adjust to obtain the pdf and then they are transformed into .txt

In [15]:
document_folder = './Docs/origin/'
try:
    pdf_files = [os.path.join(document_folder, f) for f in os.listdir(document_folder) if f.endswith('.pdf')]
    txt_files = [os.path.join(document_folder, f) for f in os.listdir(document_folder) if f.endswith('.txt')]
    documents_files = pdf_files + txt_files
except:
    raise ValueError("files don't found")

In [16]:
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with open(pdf_path, 'rb') as pdf_file:
            reader = PyPDF2.PdfReader(pdf_file)
            for page in reader.pages:
                text += page.extract_text()
        
       # Clean and process the text to remove unnecessary line breaks
        text = " ".join(text.splitlines())  # Combine the lines into a single paragraph
        text = " ".join(text.split())      # Remove multiple spaces
    except Exception as e:
        print(f"Error processing {pdf_path}: {e}")
    return text

for file_path in documents_files:
    if file_path.endswith('.pdf'):
        print(f"Processing PDF file: {file_path}")
        extracted_text = extract_text_from_pdf(file_path)
        print(f"Extracted content (first 50 characters):\n{extracted_text[:50]}")
    elif file_path.endswith('.txt'):
        print(f"TXT file found: {file_path}")
        # Read the contents of the TXT file
        with open(file_path, 'r', encoding='utf-8') as txt_file:
            text = txt_file.read() 
        print(f"Extracted content (first 50 characters)::\n{text[:50]}")

Processing PDF file: ./Docs/origin/1.pdf
Extracted content (first 50 characters):
Site Reliability Engineering at Google Taken from:


In [17]:
input_folder = "./Docs/extraidos"
for file_path in documents_files:
    extracted_text = extract_text_from_pdf(file_path)
    output_file = os.path.join(input_folder, os.path.basename(file_path).replace('.pdf', '.txt'))
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(extracted_text)
    print(f"Text saved in: {output_file}")


Text saved in: ./Docs/extraidos\1.txt


# text processing

In [18]:
genai.configure(api_key=API_KEY_GOOGLE)

model = genai.GenerativeModel("gemini-1.5-flash")

output_folder = "./Docs/pre-procesamiento"  # Carpeta para guardar resultados
os.makedirs(output_folder, exist_ok=True)

# Función para procesar un archivo .txt con el modelo generativo
def process_txt_file(file_path):
    try:
        # Leer el contenido del archivo .txt
        with open(file_path, 'r', encoding='utf-8') as f:
            raw_text = f.read()

        # Prompt para estructurar el contenido
        prompt = (
            "Organize and improve the following text into structured sections: "
            "Introduction, Development, and Conclusion. Ensure it is clean, "
            "detailed, and suitable for educational purposes:\n\n"
            f"{raw_text}"
        )

        # Generar contenido con el modelo
        response = model.generate_content([prompt])
        return response.text
    except Exception as e:
        print(f"Error procesando {file_path}: {e}")
        return None

# Procesar cada archivo .txt en la carpeta de entrada
for file_name in os.listdir(input_folder):
    if file_name.endswith('.txt'):
        input_path = os.path.join(input_folder, file_name)
        output_path = os.path.join(output_folder, file_name)

        print(f"Procesando: {file_name}")
        processed_text = process_txt_file(input_path)

        # Guardar el resultado si no hay errores
        if processed_text:
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(processed_text)
            print(f"Generación guardada en: {output_path}")
        else:
            print(f"No se pudo procesar el archivo: {file_name}")


Procesando: 1.txt
Generación guardada en: ./Docs/pre-procesamiento\1.txt


In [19]:
input_folder = "./Docs/pre-procesamiento"  # Carpeta con los textos preprocesados
output_folder = "./Docs/transcripciones_educativas"  # Carpeta para guardar transcripciones
os.makedirs(output_folder, exist_ok=True)

# Duración de la clase (opciones: '30' o '60' minutos)
lecture_duration = "60"  # Cambiar a "60" si se desea una clase más extensa

# Función para generar la transcripción educativa
def create_educational_transcription(file_path):
    try:
        # Leer el contenido del archivo preprocesado
        with open(file_path, 'r', encoding='utf-8') as f:
            preprocessed_text = f.read()

        # Prompt para generar la transcripción educativa
        prompt = (
            f"Using the following text, create a detailed and structured transcript suitable for a "
            f"{lecture_duration}-minute lecture. The output should include: \n"
            "1. An introduction to the topic.\n"
            "2. A detailed explanation of key points, with examples.\n"
            "3. A conclusion with reflections or actionable steps for students.\n\n"
            "Additionally, add a section at the end titled 'Activities' with tasks or questions "
            "to engage students and reinforce their understanding of the topic.\n\n"
            f"Input Text:\n{preprocessed_text}"
        )

        # Generar contenido con el modelo
        response = model.generate_content([prompt])
        return response.text
    except Exception as e:
        print(f"Error procesando {file_path}: {e}")
        return None

# Procesar cada archivo preprocesado en la carpeta de entrada
for file_name in os.listdir(input_folder):
    if file_name.endswith('.txt'):
        input_path = os.path.join(input_folder, file_name)
        output_path = os.path.join(output_folder, file_name)

        print(f"Generando transcripción educativa para: {file_name}")
        educational_transcription = create_educational_transcription(input_path)

        # Guardar el resultado si no hay errores
        if educational_transcription:
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(educational_transcription)
            print(f"Transcripción guardada en: {output_path}")
        else:
            print(f"No se pudo generar la transcripción para: {file_name}")


Generando transcripción educativa para: 1.txt
Transcripción guardada en: ./Docs/transcripciones_educativas\1.txt


In [20]:
# Configuración de la duración (elige entre "30" o "60")
duration = "30"  # Cambia a "60" para una clase de 60 minutos

# Prompt base según duración
base_prompt = {
    "30": (
        "Using the following text, create a detailed and structured transcript suitable for a 30-minute lecture. "
        "The output must include:\n"
        "1. An engaging introduction to the topic.\n"
        "2. Detailed explanations of key points, supported by examples.\n"
        "3. A conclusion with reflections and actionable next steps for students.\n"
        "4. Activities for students, such as questions, practical exercises, or case studies.\n\n"
        "Text:\n{content}"
    ),
    "60": (
        "Using the following text, create a detailed and structured transcript suitable for a 60-minute lecture. "
        "The output must include:\n"
        "1. An engaging and comprehensive introduction to the topic.\n"
        "2. Thorough explanations of key points, supported by detailed examples and case studies.\n"
        "3. A conclusion with reflections, future learning paths, and actionable next steps for students.\n"
        "4. Activities for students, including in-depth questions, group exercises, or real-world case studies.\n\n"
        "Text:\n{content}"
    ),
}

# Función para generar transcripciones educativas
def create_educational_transcription(file_path, duration):
    try:
        # Leer contenido del archivo preprocesado
        with open(file_path, 'r', encoding='utf-8') as f:
            preprocessed_text = f.read()

        # Crear el prompt basado en la duración
        prompt = base_prompt[duration].format(content=preprocessed_text)

        # Generar contenido con el modelo
        response = model.generate_content([prompt])
        return response.text
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None

# Procesar cada archivo preprocesado
for file_name in os.listdir(input_folder):
    if file_name.endswith('.txt'):
        input_path = os.path.join(input_folder, file_name)
        output_path = os.path.join(output_folder, file_name)

        print(f"Generating educational transcript for: {file_name} ({duration}-minute lecture)")
        educational_transcription = create_educational_transcription(input_path, duration)

        # Guardar resultado
        if educational_transcription:
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(educational_transcription)
            print(f"Transcript saved to: {output_path}")
        else:
            print(f"Failed to generate transcript for: {file_name}")


Generating educational transcript for: 1.txt (30-minute lecture)
Transcript saved to: ./Docs/transcripciones_educativas\1.txt
